# Designing Adaptive Experiments to Study Working Memory

This tutorial is based on the official documentation. You can find more details [here](http://pyro.ai/examples/intro_long.html)

In most of machine learning, we begin with data and go on to learn a model. In other contexts, we also have a hand in the data generation process. This gives us an exciting opportunity: we can try to obtain data that will help our model learn more effectively. This procedure is called *optimal experimental design* (OED) and Pyro supports choosing optimal designs through the `pyro.contrib.oed` module.

When using OED, the data generation and modelling works as follows:

1. Write down a Bayesian model involving a design parameter, an unknown latent variable and an observable.
1. Choose the optimal design (more details on this later).
1. Collect the data and fit the model, e.g. using `SVI`.

We can also run multiple 'rounds' or iterations of experiments. When doing this, we take the learned model from step 3 and use it as our prior in step 1 for the next round. This approach can be particularly useful because it allows us to design the next experiment based on what has already been learned: the experiments are *adaptive*.

In this tutorial, we work through a specific example of this entire OED procedure with multiple rounds. We will show how to design adaptive experiments to learn a participant's working memory capacity. The design we will be adapting is the *length of a sequence of digits that we ask a participant to remember*. Let's dive into the full details.


### The experiment set-up
Suppose you, the participant, are shown a sequence of digits

$$ 1\ 4\ 7\ 0\ 9 $$

which are then hidden. You have to to reproduce the sequence exactly from memory. In the next round, the length of the sequence may be different

$$ 6\ 5\ 0\ 2\ 8\ 0 .$$

The longest sequence that you can remember is your working memory capacity. In this tutorial, we build a Bayesian model for working memory, and use it to run an adaptive sequence of experiments that very quickly learn someone's working memory capacity.


### Definition of experimental units and data collection

First, we are going to define an experimental unit (i.e. our participant) and then collect the data.

Last time we worked with the working memory example, we only had one parameter to adjust. That was $\theta$ and it defined for which sequence length our participant had a probability of 50% to answer correctly.

Now the additional parameter `sensitivity` is introduced. This one defines the steepness of the slope meaning the how strong the probability changes between the sequence lengths. In the first run through the notebook we will keep this parameter fixed to `1` for all participants.

This time we are taking the `binomial_ground_truth` as the unit's ground truth. This one returns directly a response being either `1` (correct) or `0` (wrong). The parameters are given in in this format `(theta, sensitivity)`.

In [ ]:
import numpy as np
import sys
sys.path.append('..')   
from resources.synthetic import binomial_ground_truth

np.random.seed(42)

sequence_lengths = np.arange(3, 15)  # possible sequence lenghts i.e. possible experiment designs

sensitivity = 1
theta = np.random.uniform(4, 9, size=(1, 1))

Let's test the participant in this section by calling the `binomial_ground_truth()` and giving in some sequence lengths and observing the results

In [ ]:
# ADD YOUR CODE HERE

Does it work? Great, then we can move on the really interesting part!

## A model of working memory
Our model for a single round of the digits experiment described above has three components: the length $l$ of the sequence that the participant has to remember, the participant's true working memory capacity $\theta$, and the outcome of the experiment $y$ which indicates whether they were able to remember the sequence successfully ($y=1$) or not ($y=0$). We choose a prior for working memory capacity based on the (in)famous "The magical number seven, plus or minus two" [1].

**Note**: $\theta$ actually represents the point where the participant has a 50/50 chance of remembering the sequence correctly.

In [ ]:
import torch

import pyro
import pyro.distributions as dist

torch.manual_seed(42)

prior_mean = torch.tensor(7.0)
prior_sd = torch.tensor(2.0)

def model(l):
    # Dimension -1 of `l` represents the number of rounds
    # Other dimensions are batch dimensions: we indicate this with a plate_stack
    with pyro.plate_stack("plate", l.shape[:-1]):
        
        # SAMPLE THETA FROM A NORMAL DISTRIBUTION USING PYRO.SAMPLE AND THE DEFINED PRIORS
        theta = None
        
        # Share theta across the number of rounds of the experiment
        # This represents repeatedly testing the same participant
        theta = theta.unsqueeze(-1)
        # This define a *logistic regression* model for y
        logit_p = sensitivity * (theta - l)
        # The event shape represents responses from the same participant
        y = pyro.sample("y", dist.Bernoulli(logits=logit_p).to_event(1))
        return y

The probability of successfully remembering the sequence is plotted below, for five random samples of $\theta\sim p(\theta)$.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams.update({'font.size': 22})

# Sample five times from the prior distribution defined above
# ADD YOUR CODE HERE
theta_prior = None

# This is the same as using 'logits=' in the prior above
prob = torch.sigmoid(torch.Tensor(sensitivity * (theta_prior - sequence_lengths)))

plt.figure(figsize=(12, 8))
for curve in torch.unbind(prob, 0):
    plt.plot(sequence_lengths, curve.numpy(), marker='o')
plt.xlabel("Length of sequence $l$")
plt.ylabel("Probability of correctly remembering\na sequence of length $l$")
plt.legend(["Sampled person {}".format(i+1) for i in range(5)])
plt.show()

## Inference in the model

With the model in hand, we quickly demonstrate variational inference in Pyro for this model. We define a Normal guide for variational inference.

Define here the `posterior_mean` and `posterior_sd` as `pyro.params`. Copy the priors defined above with the `torch.Tensor.clone()` method.
And then sample `theta` from a Normal distribution using `dist.Normal` and passing the arguments `posterior_mean` and `posterior_sd`.

In [ ]:
from torch.distributions.constraints import positive

def guide(l):
    # The guide is initialised at the prior
    # ADD YOUR CODE HERE

We finally specify the following initial experiment design: the participant is shown sequences of lengths 3, 5, 7 and 9.

Then we collect the results form the `binomial_ground_truth()` method.

In [ ]:
# ADD YOUR CODE HERE
l_data = None
y_data = None

print(f"tested sequence lengths: {l_data}")
print(f"given responses: {y_data}")

Did you give in the correct sequences?
How do your initial responses look like?

You could take any initial sequence, of course. This was just one random guess.

We can now run SVI on the model.

Follow the steps and add your code at the respective spots.

In [ ]:
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

# rewrite the 'l_data' and 'y_data' variables into torch.Tensor objects with 'dtype=torch.float32'. 
# Use the torch.tensor() method and pass the array and dtype as arguments
# ADD YOUR CODE HERE
l_data = None
y_data = None 


assert isinstance(l_data, torch.Tensor), "Did you forget to transform the 'l_data' variable?"
assert isinstance(y_data, torch.Tensor), "Did you forget to transform the 'y_data' variable?"

# use the 'pyro.condition()' method to create a 'conditioned_model' by passing the model and the responses
# the respones are collected in a dictionary where the key is the response name and the value is the actual response array 
# ADD YOUR CODE HERE
conditioned_model = None

# we pass now everything we created so far to the SVI
svi = SVI(conditioned_model,
          guide,
          Adam({"lr": .001}),
          loss=Trace_ELBO(),
          num_samples=100)

pyro.clear_param_store()

# And now we run the Variational Inference
num_iters = 5000
for i in range(num_iters):
    elbo = svi.step(l_data)
    if i % 500 == 0:
        print("Neg ELBO:", elbo)

How does the ELBO behave over the iterations? How would you expect it to behave? Recall the optimization objective of Variational Inference!

In [ ]:
print("Prior:     N({:.3f}, {:.3f})".format(prior_mean, prior_sd))
print("Posterior: N({:.3f}, {:.3f})".format(pyro.param("posterior_mean"),
                                            pyro.param("posterior_sd")))

## Bayesian optimal experimental design
So far so standard. In the previous example, the lengths `l_data` were not chosen with a great deal of forethought. Fortunately, in a setting like this, it is possible to use a more sophisticated strategy to choose the sequence lengths to make the most of every question we ask.

Under our posterior, we can see that we have an updated estimate for the participant's working memory capacity, and our uncertainty has now decreased.

We do this using Bayesian optimal experimental design (BOED). In BOED, we are interested in designing experiments that maximise the information gain, which is defined formally as

$$ \text{IG}(l, y) = KL(p(\theta|y,l)||p(\theta)) .$$

where $KL$ represents the [Kullback-Leiber divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence).

In words, the information gain is the KL divergence from the posterior to the prior. It therefore represents the distance we "move" the posterior by running an experiment with length $l$ and getting back the outcome $y$.

Unfortunately, we will not know $y$ until we actually run the experiment. Therefore, we choose $l$ on the basis of the *expected* information gain [2]

$$\text{EIG}(l) = \mathbb{E}_{y\sim p(y|\theta,l)} [KL(p(\theta|y,l)||p(\theta))]. $$

Because it features the posterior density $p(y|\theta,l)$, the EIG is not immediately tractable. However, we can make use of the following variational estimator for EIG [3]

$$\text{EIG}(l) = \min_q \mathbb{E}_{\theta,y \sim p(\theta)p(y|\theta,l)} \left[ \log \frac{p(y|\theta,l)}{q(y|l)} \right].$$

### Optimal experimental design in Pyro
Fortunately, Pyro comes ready with tools to estimate the EIG. All we have to do is define the "marginal guide" $q(y|l)$ in the formula above.

In [ ]:
def marginal_guide(design, observation_labels, target_labels):
    # This shape allows us to learn a different parameter for each candidate design l
    q_logit = pyro.param("q_logit", torch.zeros(design.shape[-2:]))
    pyro.sample("y", dist.Bernoulli(logits=q_logit).to_event(1))

This is not a guide for inference, like the guides normally encountered in Pyro and used in `SVI`. Instead, this guide samples *only* the observed sample sites: in this case `"y"`. This makes sense because conventional guides approximate the posterior $p(\theta|y, l)$ whereas our guide approximates the marginal $p(y|l)$.

In [ ]:
from pyro.contrib.oed.eig import marginal_eig

# The shape of `candidate_designs` is (number designs, 1)
# This represents a batch of candidate designs, each design is for one round of experiment
candidate_designs = torch.tensor(sequence_lengths, dtype=torch.float32).reshape(-1, 1)

pyro.clear_param_store()

num_steps, start_lr, end_lr = 1000, 0.1, 0.001
optimizer = pyro.optim.ExponentialLR({'optimizer': torch.optim.Adam,
                                      'optim_args': {'lr': start_lr},
                                      'gamma': (end_lr / start_lr) ** (1 / num_steps)})

eig = marginal_eig(model,
                   candidate_designs,       # design, or in this case, tensor of possible designs
                   "y",                     # site label of observations, could be a list
                   "theta",                 # site label of 'targets' (latent variables), could also be list 
                   num_samples=100,         # number of samples to draw per step in the expectation
                   num_steps=num_steps,     # number of gradient steps
                   guide=marginal_guide,    # guide q(y)
                   optim=optimizer,         # optimizer with learning rate decay
                   final_num_samples=10000  # at the last step, we draw more samples for a more accurate EIG estimate
                  )

We can visualize the EIG estimates that we found.

In [ ]:
plt.figure(figsize=(10,5))
matplotlib.rcParams.update({'font.size': 22})
plt.plot(candidate_designs.numpy(), eig.detach().numpy(), marker='o', linewidth=2)
plt.xlabel("$l$")
plt.ylabel("EIG($l$)")
plt.title(f"Expected Information Gain")
plt.show()

Now we can get the experiment design which is expected to yield the highest information gain.

Get the best suited candidate from the `candidate_designs` according to the EIG estimate.

In [ ]:
# ADD YOUR CODE HERE
best_l = None

This tells us that the first round should be run with a sequence of length 7. Note that, while we might have been able to guess this optimal design intuitively, this same framework applies equally well to more sophisticated models and experiments where finding the optimal design by intuition is more challenging.

As a side-effect of training, our marginal guide $q(y|l)$ has approximately learned the marginal distribution $p(y|l)$.

In [ ]:
q_prob = torch.sigmoid(pyro.param("q_logit"))
print("   l | q(y = 1 | l)")
for (sequence_lengths, q) in zip(candidate_designs, q_prob):
    print("{:>4} | {}".format(int(sequence_lengths.item()), q.item()))

The elements of this fitted tensor represent the marginal over $y$, for each possible sequence length $l$ in `candidate_designs`. We have marginalised out the unknown $\theta$ so this fitted tensor shows the probabilities for an 'average' participant.

## The adaptive experiment

We now have the ingredients to build an adaptive experiment to study working memory. We repeat the following steps:

1. Use the EIG to find the optimal sequence length $l$
2. Run the test using a sequence of length $l$
3. Update the posterior distribution with the new data


At the first iteration, step 1 is done using the prior as above. However, for subsequent iterations, we use the posterior given all the data so far.

In this section, the "experiment" is performed using the defined ground truth from above but make sure to return a `torch.Tensor` with `dtype=torch.float32` from the responses. This is done by passing the response to the `torch.tensor()` method and define the argument `dtype`. 

In [ ]:
def participant(l):
    return torch.tensor(binomial_ground_truth(l, (theta, sensitivity)), dtype=torch.float32)

The following code allows us to update the model as we gather more data.

In [ ]:
def make_model(mean, sd):
    def model(l):
        # Dimension -1 of `l` represents the number of rounds
        # Other dimensions are batch dimensions: we indicate this with a plate_stack
        with pyro.plate_stack("plate", l.shape[:-1]):
            theta = pyro.sample("theta", dist.Normal(mean, sd))
            # Share theta across the number of rounds of the experiment
            # This represents repeatedly testing the same participant
            theta = theta.unsqueeze(-1)
            # This define a *logistic regression* model for y
            logit_p = sensitivity * (theta - l)
            # The event shape represents responses from the same participant
            y = pyro.sample("y", dist.Bernoulli(logits=logit_p).to_event(1))
            return y
    return model

Now we have everything to run a 10-step experiment using adaptive designs.

In [ ]:
ys = torch.tensor([])
ls = torch.tensor([])

history = [(prior_mean, prior_sd)]

pyro.clear_param_store()

current_model = make_model(prior_mean, prior_sd)

for experiment in range(10):
    print("Round", experiment + 1)
     
    # Step 1: compute the optimal length
    optimizer = pyro.optim.ExponentialLR({'optimizer': torch.optim.Adam,
                                          'optim_args': {'lr': start_lr},
                                          'gamma': (end_lr / start_lr) ** (1 / num_steps)})
    eig = marginal_eig(
        current_model, 
        candidate_designs, 
        "y", "theta", 
        num_samples=100,
        num_steps=num_steps, 
        guide=marginal_guide, 
        optim=optimizer,
        final_num_samples=10000
        )
    
    best_l = candidate_designs[torch.argmax(eig)].float().detach()
    
    # Step 2: run the experiment, here using the synthetic person
    print("Asking the participant to remember a sequence of length", int(best_l.item()))
    y = participant(best_l.item())
    if y:
        print("Participant remembered correctly")
    else:
        print("Participant could not remember the sequence")
    # Store the sequence length and outcome
    ls = torch.cat([ls, best_l.expand(1)], dim=0)
    ys = torch.cat([ys, y.expand(1)])
    
    # Step 3: learn the posterior using all data seen so far
    conditioned_model = pyro.condition(model, {"y": ys})
    
    svi = SVI(
        conditioned_model,
        guide,
        Adam({"lr": .005}),
        loss=Trace_ELBO(),
        num_samples=100
        )
    
    num_iters = 2000
    for i in range(num_iters):
        elbo = svi.step(ls)
        
    history.append((pyro.param("posterior_mean").detach().clone().numpy(),
                    pyro.param("posterior_sd").detach().clone().numpy()))
    current_model = make_model(pyro.param("posterior_mean").detach().clone(),
                               pyro.param("posterior_sd").detach().clone())
    print("Estimate of \u03b8: {:.3f} \u00b1 {:.3f}\n".format(*history[-1]))

Now let's visualize the evolution of the posterior over $\theta$:

In [ ]:
import numpy as np
from scipy.stats import norm
import matplotlib.colors as colors
import matplotlib.cm as cmx


matplotlib.rcParams.update({'font.size': 22})
cmap = plt.get_cmap('winter') 
cNorm  = colors.Normalize(vmin=0, vmax=len(history)-1)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=cmap)
plt.figure(figsize=(12, 6))
x = np.linspace(0, 14, 100)
for idx, (mean, sd) in enumerate(history):
    color = scalarMap.to_rgba(idx)
    y = norm.pdf(x, mean, sd)
    plt.plot(x, y, color=color)
    plt.xlabel("$\\theta$")
    plt.ylabel("p.d.f.")
plt.show()

(Blue = prior, light green = 10 step posterior)

By contrast, suppose we use a simplistic design: try the first 10 sequence lengths.

In [ ]:
pyro.clear_param_store()
ls = candidate_designs[:10].reshape(-1,)
ys = [torch.tensor(binomial_ground_truth(l.item(), (theta[0], sensitivity)), dtype=torch.float32) for l in ls]
ys = torch.tensor(ys, dtype=torch.float32)
conditioned_model = pyro.condition(model, {"y": ys})
svi = SVI(conditioned_model,
          guide,
          Adam({"lr": .005}),
          loss=Trace_ELBO(),
          num_samples=100)
num_iters = 2000
for i in range(num_iters):
    elbo = svi.step(ls)

In [ ]:
plt.figure(figsize=(12,6))
matplotlib.rcParams.update({'font.size': 22})
y1 = norm.pdf(x, pyro.param("posterior_mean").detach().numpy(),
              pyro.param("posterior_sd").detach().numpy())
y2 = norm.pdf(x, history[-1][0], history[-1][1])
plt.plot(x, y1)
plt.plot(x, y2)
plt.legend(["Simple design", "Optimal design"])
plt.xlabel("$\\theta$")
plt.ylabel("p.d.f.")
plt.show()

Although both design strategies give us data, the optimal strategy ends up with a posterior distribution that is more peaked: that means we have greater confidence in our final answer, or may be able to stop experimenting earlier.

## Additional tasks

Did the tutorial help you to understand how Variational Inference and Pyro can be utilized for BOED? The following paragraphs give you a guide line for more tasks. These will help you to understand the effect of different parameters and model complexity on the obtained result. Make for each task another copy of the notebook.

### Adjusting the experiment and SVI
Starting with the prior, we had a pretty informed guess with `prior_mean=7`. In fact, when running experiments on new hypothesis or more complex cognitive mechanisms with more complex latent variables it's quite difficult to make such informed guesses. What would happen if you would make a "worse" guess? Let your prior be chosen by e.g. a Uniform distribution instead of a Normal and re-run the code. go ahead and try to break the algorithm - this will give you an impression on the importance of the prior ;) 

What happens if the result does not look good? Can you can still get a good result? 

You could for example adjust the experiment setup by increasing the number of rounds, the amount of initial guesses and for the SVI you could increase the number of samples and the number of iterations. 

Go ahead and try different parameter combinations to get a feeling for how they influence the obtained result!

### Adjusting the model
The model in this tutorial is very simple and could be extended in a number of ways. For instance, we could let the `sensitivity` parameter be guessed additonally to the working memory capacity `theta`. 
Further, it's also possible that as well as measuring whether the participant did or did not remember the sequence, we might collect some other information as well. We could build a model for the number of mistakes made (e.g. the edit distance between the correct sequence and the participant's response) or jointly model the correctness and the time taken to respond. Here is an example model where we model the response time using a LogNormal distribution, as suggested by [4]. You can obtain the response time from the `binomial_ground_truth()` by setting the argument `response_time=True`. Run an SVI to obtain a model for the new experiment and compare the response time distributions obtained once by the ground truth and another time by the optimized model. You can use the `plt.hist()` method by `matplotlib`. 

### Trying different sampling strategies
In the research of experimental design and in research in general, it's always important to compare your findings by baselines and other state-of-the-art procedures to have a meaningful result.  

Until now, we analysed only the BOED for this experimental setup. Compare its performance with other sampling algorithms which you already know from previous tutorials. Try them for a simple ground truth with only one model parameter (`theta`) for two parameters (`theta` and `sensitivity`) and for mulitple observations (`response` and `response time`).

### Optional: Adjusting the posterior inference strategy
In this tutorial we used variational inference to fit an approximate posterior for $\theta$. This could be substituted for an alternative posterior inference strategy, such as Hamiltonian Monte Carlo.

In [ ]:
time_intercept = 0.5
time_scale = 0.5

def model(l):
    theta = pyro.sample("theta", dist.Normal(prior_mean, prior_sd))
    logit_p = sensitivity * (theta - l)
    correct = pyro.sample("correct", dist.Bernoulli(logits=logit_p))
    mean_log_time = time_intercept + time_scale * (theta - l)
    time = pyro.sample("time", dist.LogNormal(mean_log_time, 1.0))
    return correct, time

It would still be possible to compute the EIG using `marginal_eig`. We would replace `"y"` by `["correct", "time"]` and the marginal guide would now model a joint distribution over the two sites `"correct"` and `"time"`.

We could also think about learning at two levels: learning global variables for population trends as well as local variables for individual level effects. The current model is an individual only model. The EIG could still be used as a means to select the optimal design in such scenarios.

# Congratulations!

You finished the last tutorial of the Optimizing Experimental Design course!

I hope you learned a lot about the different sampling strategies and most importantly had fun!

## References

[1] Miller, G.A., 1956. **The magical number seven, plus or minus two: Some limits on our capacity for processing information.** Psychological review, 63(2), p.81.

[2] Chaloner, K. and Verdinelli, I., 1995. **Bayesian experimental design: A review.** Statistical Science, pp.273-304.

[3] Foster, A., Jankowiak, M., Bingham, E., Horsfall, P., Teh, Y.W., Rainforth, T. and Goodman, N., 2019. **Variational Bayesian Optimal Experimental Design.** Advances in Neural Information Processing Systems 2019 (to appear).

[4] van der Linden, W.J., 2006. **A lognormal model for response times on test items.** Journal of Educational and Behavioral Statistics, 31(2), pp.181-204.